In [5]:
import sys
import csv
import tweepy
import re 
import nltk
import string
from nltk.classify import *
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from nltk.corpus import stopwords
import nltk.classify.util

#initialize stopWords
 
#starting the function 
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end
 
#starting the function 
#end
 
#starting the function 
def getFeatureVector(tweet):
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        featureVector.append(w.lower())
    return featureVector
#end
 
#starting the function 
def featureExtraction():
    #Here I am reading the tweets one by one and process it
    inpTweets = csv.reader(open('ManuallyTag_naive.csv', 'rt'), delimiter=',', quotechar='|')
    tweets = []
  
    for rowTweet in inpTweets:
        sentiment = rowTweet[7]
        tweet = rowTweet[3]
        featureVector = getFeatureVector(tweet)
        tweets.append((featureVector, sentiment))
    return tweets #Here I am returning the tweets inside the array plus its sentiment
#end

tweets = featureExtraction()
#print tweets

#Classifier 
def get_words_in_tweets(tweets):
    all_words = []
    for (text, sentiment) in tweets:
        all_words.extend(text)
    return all_words

def get_word_features(wordlist):
    
    # This line calculates the frequency distrubtion of all words in tweets
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    
    # This prints out the list of all distinct words in the text in order
    # of their number of occurrences.
    return word_features

word_features = get_word_features(get_words_in_tweets(tweets)) #my list of many words 

def extract_features(tweet):
    settweet = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in settweet)
    return features


#Here I am creating my Training set. 
#I extract feature vector for all tweets in one shot
#accuracy for the first 250 tweets
training_set = nltk.classify.apply_features(extract_features, tweets[101:])
test_set = nltk.classify.apply_features(extract_features, tweets[:100])

#****** Naive Bayes Classifier******************************************

classifier = nltk.NaiveBayesClassifier.train(training_set)

# Accuracy
accuracy = nltk.classify.accuracy(classifier, training_set) 

#Printing the accuracy
print (accuracy) 

total = accuracy * 100 
print ('Naive Bayes Accuracy: %4.2f' % total) 

 






0.9230769230769231
Naive Bayes Accuracy: 92.31
